In [1]:
from torchvision.transforms.v2 import ToTensor

VERSION = '001'
BATCHSIZE = 32
MAX_EPOCHS = 50
BACKBONE = "resnet50"
MODEL_NAME = "ResNet50_transfer"
NUM_WORKERS = 31

In [2]:

import os
from torchvision.transforms import Compose, Resize, InterpolationMode, ToTensor, Normalize
import torch
from torchvision import transforms
from code.classifier import PneumoniaClassifier, Config



In [3]:

# Load the TensorBoard extension
%reload_ext tensorboard
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
# Enable CUDA Launch Blocking
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# Set float32 matrix multiplication precision to 'medium' to utilize Tensor Cores
torch.set_float32_matmul_precision('medium')
torch.serialization.add_safe_globals([Compose, Resize, set, Config, InterpolationMode, ToTensor, Normalize])

In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Thu Nov 21 08:54:02 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.90                 Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 Ti   WDDM  |   00000000:02:00.0  On |                  N/A |
|  0%   61C    P2             64W /  285W |   11319MiB /  12282MiB |     99%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

config = Config(
    backbone_name="resnet50",
    transfer_learning=True,
    learning_rate=1e-4,
    batch_size=20,
    max_epochs=100,
    weight_decay=1e-4,
    dropout=0.5,
    num_workers=31,
    model_name="ResNet50_transfer",
    version="001",
    optimizer_name = "sgd"
)

model = PneumoniaClassifier(config, transform)
model.train_model()


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Users\aszab\Documents\Studium\Modules\ComputerScienceProject\MedicalImaging\pneumonia_detection\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(

  | Name              | Type            | Params | Mode 
--------------------------------------------------------------
0 | accuracy          | BinaryAccuracy  | 0      | train
1 | precision         | BinaryPrecision | 0      | train
2 | recall            | BinaryRecall    | 0      | train
3 | f

Epoch 0: 100%|██████████| 261/261 [07:58<00:00,  0.55it/s, v_num=7]        
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 261/261 [08:00<00:00,  0.54it/s, v_num=7, val_acc_epoch=0.711, val_precision_epoch=0.746, val_recall_epoch=0.926, val_f1_epoch=0.826]

C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryAccuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryPrecision was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryRecall was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
C:\Users\aszab\miniconda3\env

Epoch 1:  41%|████▏     | 108/261 [03:03<04:20,  0.59it/s, v_num=7, val_acc_epoch=0.711, val_precision_epoch=0.746, val_recall_epoch=0.926, val_f1_epoch=0.826, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [6]:
checkpoint = torch.load(config.model_name, weights_only=True)

# Access the metadata
metadata = checkpoint.get("metadata", {})

FileNotFoundError: [Errno 2] No such file or directory: '../checkpoints/ResNet50_transfer-epoch=08-val_loss=0.15_v001.ckpt'

In [20]:
# Initialize the model
model = PneumoniaClassifier(config, transform)
# Test the model
metadata = model.test_model(config.model_name)

# Print the metadata
print(metadata)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 20/20 [00:18<00:00,  1.10it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_acc_epoch         0.48076921701431274
      test_acc_step         0.6751185059547424
      test_f1_epoch         0.38167938590049744
      test_f1_step          0.2445952147245407
        test_loss           0.7132286429405212
  test_precision_epoch       0.746268630027771
   test_precision_step      0.37489965558052063
    test_recall_epoch       0.25641027092933655
    test_recall_step        0.19538259506225586
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
None


In [7]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

config = Config(
    backbone_name="resnet50",
    transfer_learning=False,
    learning_rate=1e-4,
    batch_size=20,
    max_epochs=100,
    weight_decay=1e-4,
    dropout=0.5,
    num_workers=31,
    model_name="ResNet50",
    version="001",
    optimizer_name = "sgd"
)

model = PneumoniaClassifier(config, transform)
model.train_model()

checkpoint = torch.load(config.model_name, weights_only=True)

# Access the metadata
metadata = checkpoint.get("metadata", {})

# Initialize the model
model = PneumoniaClassifier(config, transform)
# Test the model
metadata = model.test_model(config.model_name)

# Print the metadata
print(metadata)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type            | Params | Mode 
--------------------------------------------------------------
0 | accuracy          | BinaryAccuracy  | 0      | train
1 | precision         | BinaryPrecision | 0      | train
2 | recall            | BinaryRecall    | 0      | train
3 | f1                | BinaryF1Score   | 0      | train
4 | feature_extractor | Sequential      | 23.5 M | train
5 | dropout           | Dropout         | 0      | train
6 | classifier        | Linear          | 4.1 K  | train
--------------------------------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params
94.049    Total estimated model params size (MB)
156       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


Epoch 1:  41%|████▏     | 108/261 [04:12<05:57,  0.43it/s, v_num=7, val_acc_epoch=0.711, val_precision_epoch=0.746, val_recall_epoch=0.926, val_f1_epoch=0.826, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]


NameError: name 'exit' is not defined

In [ ]:
transform = transforms.Compose([
    transforms.Resize((1000, 1000)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

config = Config(
    backbone_name="resnet50",
    transfer_learning=True,
    learning_rate=1e-4,
    batch_size=20,
    max_epochs=100,
    weight_decay=1e-4,
    dropout=0.5,
    num_workers=31,
    model_name="ResNet50_transfer_1k",
    version="001",
    optimizer_name = "sgd"
)

model = PneumoniaClassifier(config, transform)
model.train_model()

checkpoint = torch.load(config.model_name, weights_only=True)

# Access the metadata
metadata = checkpoint.get("metadata", {})

# Initialize the model
model = PneumoniaClassifier(config, transform)
# Test the model
metadata = model.test_model(config.model_name)

# Print the metadata
print(metadata)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((1000, 1000)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

config = Config(
    backbone_name="resnet50",
    transfer_learning=False,
    learning_rate=1e-4,
    batch_size=20,
    max_epochs=100,
    weight_decay=1e-4,
    dropout=0.5,
    num_workers=31,
    model_name="ResNet50_1k",
    version="001",
    optimizer_name = "sgd"
)

model = PneumoniaClassifier(config, transform)
model.train_model()

checkpoint = torch.load(config.model_name, weights_only=True)

# Access the metadata
metadata = checkpoint.get("metadata", {})

# Initialize the model
model = PneumoniaClassifier(config, transform)
# Test the model
metadata = model.test_model(config.model_name)

# Print the metadata
print(metadata)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((1000, 1000)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

config = Config(
    backbone_name="resnet18",
    transfer_learning=True,
    learning_rate=1e-4,
    batch_size=20,
    max_epochs=100,
    weight_decay=1e-4,
    dropout=0.5,
    num_workers=31,
    model_name="ResNet18_transfer",
    version="001",
    optimizer_name = "sgd"
)

model = PneumoniaClassifier(config, transform)
model.train_model()

checkpoint = torch.load(config.model_name, weights_only=True)

# Access the metadata
metadata = checkpoint.get("metadata", {})

# Initialize the model
model = PneumoniaClassifier(config, transform)
# Test the model
metadata = model.test_model(config.model_name)

# Print the metadata
print(metadata)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((1000, 1000)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

config = Config(
    backbone_name="resnet18",
    transfer_learning=False,
    learning_rate=1e-4,
    batch_size=20,
    max_epochs=100,
    weight_decay=1e-4,
    dropout=0.5,
    num_workers=31,
    model_name="ResNet18",
    version="001",
    optimizer_name = "sgd"
)

model = PneumoniaClassifier(config, transform)
model.train_model()

checkpoint = torch.load(config.model_name, weights_only=True)

# Access the metadata
metadata = checkpoint.get("metadata", {})

# Initialize the model
model = PneumoniaClassifier(config, transform)
# Test the model
metadata = model.test_model(config.model_name)

# Print the metadata
print(metadata)

In [19]:
# Start TensorBoard and point it to the log directory
%tensorboard --logdir tb_logs/ResNet50_transfer/version_2

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 39112), started 2 days, 4:06:44 ago. (Use '!kill 39112' to kill it.)

# Add Regularization